In [1]:
import torch
import numpy as np
import torch.nn as nn
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# understanding data
iris = datasets.load_iris()
x = iris.data
y = iris.target
# shuffle x and y with same order
x, y = shuffle(x, y, random_state=0)

print(x[:3,:])
print(y[:3])
print(x.shape)
print(y.shape)

[[5.8 2.8 5.1 2.4]
 [6.  2.2 4.  1. ]
 [5.5 4.2 1.4 0.2]]
[2 1 0]
(150, 4)
(150,)


In [3]:
# step 0 -> prepare data
n_samples, n_features = x.shape
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.1)

sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

# transform from numpy to tensor
x_train = torch.from_numpy(x_train.astype(np.float32))
x_test = torch.from_numpy(x_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

# reshape y_train and y_test
y_train = y_train.view(-1,1)
y_test = y_test.view(-1,1)
print(x_train.size())
print(y_train.size())

torch.Size([135, 4])
torch.Size([135, 1])


In [4]:
# step 1 -> model
class logisticRegression(nn.Module):
    def __init__(self, num_input_features):
        super(logisticRegression, self).__init__()
        self.linear = nn.Linear(num_input_features,1)
        
    def forward(self,x):
        y_hat = torch.sigmoid(self.linear(x))
        return y_hat

model = logisticRegression(n_features)

In [5]:
# step 2 -> optimiser and loss
criterion = nn.BCELoss()
learning_rate = 0.01
optimiser = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [6]:
# step 3 -> training loop
n_epochs = 100
for epoch in range(n_epochs):
    # forward pass
    y_hat = model(x_train)
    
    # loss
    loss = criterion(y_hat,y_train)
    
    # backward pass
    loss.backward()
    
    # update weights
    optimiser.step()
    
    # reload graidents to zero
    optimiser.zero_grad()
    
    if epoch%10 == 0: print(f'epoch: {epoch}, loss: {loss}')


epoch: 0, loss: 0.6325342059135437
epoch: 10, loss: 0.3439575135707855
epoch: 20, loss: 0.08671033382415771
epoch: 30, loss: -0.14045299589633942
epoch: 40, loss: -0.3414994776248932
epoch: 50, loss: -0.5214387774467468
epoch: 60, loss: -0.6850612759590149
epoch: 70, loss: -0.8363652229309082
epoch: 80, loss: -0.9784616231918335
epoch: 90, loss: -1.1136949062347412


In [7]:
with torch.no_grad():
    y_hat_test = model(x_test)
    y_test_cls = y_hat_test.round()
    acc = y_test_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(acc)

tensor(0.7333)


In [8]:
# X = np.array([[1., 0.], [2., 1.], [0., 0.]])
# y = np.array([0, 1, 2])
# from sklearn.utils import shuffle
# X, y = shuffle(X, y, random_state=0)
# print(X)
# print(y)